In [0]:
import pandas as pd
import os
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPool2D, AveragePooling2D
from sklearn.metrics import classification_report
import keras.optimizers
from keras.utils import np_utils
from sklearn import metrics 
from matplotlib import pyplot as plt
from keras.regularizers import l2
import tensorflow as tf
from keras.models import model_from_json

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Perpare data frame of Feature and Label

In [0]:
DATA_DIR = "/content/drive/My Drive/498L Project/data/"

SPECIES_LIST = ["Parus major", "Phylloscopus collybita", "Baeolophus bicolor", "Melospiza melodia", "Cardinalis cardinalis", "Hirundo rustica", "Zenaida macroura",
                "Spinus tristis", "Icterus galbula", "Sturnella magna", "Thryothorus ludovicianus", "Passer domesticus", "Poecile atricapillus", 
                "Turdus migratorius", "Cyanistes caeruleus","Loxia curvirostra", "Passer montanus", "Sylvia atricapilla", "Sylvia communis"]

df = pd.DataFrame(columns=['Feature', 'Class_Label'])
for j in range(0,len(SPECIES_LIST)):
  features = []
  species = SPECIES_LIST[j]
  tensor_file_loc = DATA_DIR + species + "/MFCC Tensor/feature_tensor.p"
  tensor = pickle.load( open( tensor_file_loc, "rb" ) )
  length = len(tensor)
  for i in range(0,length):
    row = [tensor[i], species]
    df.loc[len(df)] = row

Doing one hot encoding and splitting to train and test sets

In [0]:
X = np.array(df.Feature.tolist())
y = np.array(df.Class_Label.tolist())
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 
x_training, x_test, y_training, y_test = train_test_split(X, yy, test_size=0.15, random_state = 42)
x_train, x_val, y_train, y_val = train_test_split(x_training, y_training, test_size=0.1, random_state = 42)

Building the Model (LeNet-5)

In [0]:
num_rows = 20
num_columns = 1500
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_val = x_val.reshape(x_val.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
stride = 2
kernal = 2
# Construct model 
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=kernal, input_shape=(num_rows, num_columns, num_channels)))#, activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=2, strides=(stride,stride)))
#model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=kernal, activation='relu'))#,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=2, strides=(stride,stride)))
#model.add(Dropout(0.3))

model.add(Conv2D(filters=256, kernel_size=kernal, activation='relu'))#,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Conv2D(filters=256, kernel_size=kernal, activation='relu'))
model.add(Conv2D(filters=256, kernel_size=kernal, activation='relu'))

#model.add(Dropout(0.2))
model.add(GlobalMaxPool2D())
model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_labels, activation='softmax'))#,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.summary()

Train Model


In [0]:
# Compile the model
opt = keras.optimizers.Adam(learning_rate=0.005)
#opt = keras.optimizers.SGD(lr=0.015,momentum=0.5)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)

num_epochs = 150
num_batch_size = 200
history = model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_val, y_val), verbose=1,shuffle=True)

# Calculate accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(x_test)
print(classification_report(Y_test, y_pred))

print("Overall Accuracy: %.4f%%" % accuracy) 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Save the Model

In [0]:
model_json = model.to_json()
with open("AlexNet_Base_Model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("AlexNet_Base_Model.h5")